# Praktiskais darbs 2. Stereo attēlu dispritātes kartes un dziļuma kartes iegūšana
RTU DE0942 • Datorredze

**Mērķis:** Apgūt dziļuma informācijas ieguvi no stereo pāriem, izmantojot disparitātes aprēķinu un kameras kalibrācijas parametrus.
**Uzdevumi:**
1. Disparitātes karte bez kalibrācijas (Middlebury dati)
2. Disparitāte un dziļuma karte ar kalibrācijas datiem
3. Disparitātes un dziļuma karte ar reāliem attēliem



## prepare data

In [372]:
from google.colab import files
import cv2
import matplotlib.pyplot as plt
import numpy as np
import reprlib

import json
import time

!pip install gdown -q
!pip install piexif -q

In [373]:
import gdown
import os
import time
from urllib.parse import urlparse, parse_qs

def is_folder_empty(path):
    return not any(os.scandir(path))

def get_folder_id_from_link(shared_link):
    parts = urlparse(shared_link)
    return parts.path.split('/')[-1]

def download_folder_if_missing(shared_link, output_dir):
    folder_id = get_folder_id_from_link(shared_link)
    target_dir = output_dir #os.path.join(output_dir, folder_id) # Create a subdirectory for each link

    # Get list of files already downloaded
    downloaded_files = []
    if os.path.exists(target_dir):
        downloaded_files = os.listdir(target_dir)

    print(f" > Checking contents of shared link: {shared_link}")
    filelist = gdown.download_folder(id=folder_id, output=target_dir, quiet=True, use_cookies=False, skip_download=True)
    print(f" > {len(filelist)} files to download: {filelist}")

    # Download files individually
    missing_files = []
    for file_info in filelist:
        filename = file_info.path  # or file_info.local_path depending on structure
        file_id = file_info.id

        # Check if file already exists
        if filename not in downloaded_files:
            missing_files.append(file_info)
            print(f" > Downloading: {filename}")
            try:
                # Download individual file by ID
                output_path = os.path.join(target_dir, filename)
                gdown.download(
                    id=file_id,
                    output=output_path,
                    quiet=False,
                    use_cookies=False
                )
            except Exception as e:
                #print(f" > Error downloading {filename}: {e}")
                url = f"https://drive.google.com/uc?id={file_id}"
                print(f" > Error downloading {filename}: {url}")

shared_links = [
    'https://drive.google.com/drive/folders/1fh5Cv7nFC8W1jziJAVcWTpZZn1SK_f76?usp=sharing', #pt1
    'https://drive.google.com/drive/folders/1Ku85oKCrXJ4d-y0MYTyaaBTq4HeH6aLA?usp=sharing' #pt2
]

folder_path_root = 'calibrate2'

print("Starting download process...")

# Create the root output directory if it doesn't exist
os.makedirs(folder_path_root, exist_ok=True)


for shared_link in shared_links:
  download_folder_if_missing(shared_link, output_dir=folder_path_root)
  time.sleep(5) # Add a 5-second delay between downloads

print(f"Download process finished. Contents are in: {folder_path_root}")

# Display contents of the downloaded folder(s)
print("\nContents of root folder:")
!ls -lah $folder_path_root
!ls -1 $folder_path_root | wc -l

Starting download process...
 > Checking contents of shared link: https://drive.google.com/drive/folders/1fh5Cv7nFC8W1jziJAVcWTpZZn1SK_f76?usp=sharing
 > 37 files to download: [GoogleDriveFileToDownload(id='1oJB12nhI87kF_T7RdAUMek3f34AffRTd', path='IMG_20251107_131104.jpg', local_path='calibrate2/IMG_20251107_131104.jpg'), GoogleDriveFileToDownload(id='1iDthlPVokjhhWtSVKg9vS8MU4CqXtK2C', path='IMG_20251107_131107.jpg', local_path='calibrate2/IMG_20251107_131107.jpg'), GoogleDriveFileToDownload(id='1ZAaz_r5RYeSVh1cWhi0aqPoiPE6-akMP', path='IMG_20251107_131111.jpg', local_path='calibrate2/IMG_20251107_131111.jpg'), GoogleDriveFileToDownload(id='1DM16V_3pFuX0pPx0wMh7gXDwPszF9HLN', path='IMG_20251107_131116.jpg', local_path='calibrate2/IMG_20251107_131116.jpg'), GoogleDriveFileToDownload(id='1lDc_6_wYkC4TyYR1a9Q_L3XUumRa1tiV', path='IMG_20251107_131118.jpg', local_path='calibrate2/IMG_20251107_131118.jpg'), GoogleDriveFileToDownload(id='16tHfnY2nZz1vef5r6WJ9XY6vPMCU_X0U', path='IMG_2025110

In [374]:
def isGrayscale(img):
  return len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[2] == 1)

def plotImage(img, title , cmap = None, colorbar_title = None):
  plt.figure(figsize=(10, 4))

  if cmap is None:
    if isGrayscale(img):
      plt.imshow(img, cmap='gray') # useful for grayscale images to avoid false coloring
    else:
      plt.imshow(img)
  else:
    plt.imshow(img, cmap=cmap)

  if colorbar_title:
    plt.colorbar(label=colorbar_title)

  plt.title(title)
  plt.axis('off')
  plt.show()

def plotImgSideBySide(img1, title1, img2, title2, subtitle=None, cmap1 = None, cmap2=None, img2_vmax=None, colorbar1_title = None, colorbar2_title = None):
  fig, axs = plt.subplots(1, 2, figsize=(16, 6))

  if cmap1 is None:
    if isGrayscale(img1):
      axs[0].imshow(img1, cmap='gray') # useful for grayscale images to avoid false coloring
    else:
      axs[0].imshow(img1)
  else:
    axs[0].imshow(img1, cmap=cmap1)

  if colorbar1_title:
    fig.colorbar(axs[0].images[0], ax=axs[0], label=colorbar1_title)
  axs[0].set_title(title1)
  axs[0].axis('off')


  if cmap2 is None:
    if isGrayscale(img2):
      axs[1].imshow(img2, cmap='gray', vmax=img2_vmax) # useful for grayscale images to avoid false coloring
    else:
      axs[1].imshow(img2, vmax=img2_vmax)
  else:
    axs[1].imshow(img2, cmap=cmap2, vmax=img2_vmax)

  if colorbar2_title:
    fig.colorbar(axs[1].images[0], ax=axs[1], label=colorbar2_title)
  axs[1].set_title(title2)
  axs[1].axis('off')

  if subtitle is not None:
    fig.suptitle(subtitle, fontsize=16)
  plt.show()

def plotImgSideBySideTriple(img1, title1, img2, title2, img3, title3):
  plt.figure(figsize=(15, 5))

  plt.subplot(1, 3, 1)
  plt.title(title1)
  if isGrayscale(img1):
    plt.imshow(img1, cmap='gray') # useful for grayscale images to avoid false coloring
  else:
    plt.imshow(img1)
  plt.axis('off')

  plt.subplot(1, 3, 2)
  plt.title(title2)
  if isGrayscale(img2):
    plt.imshow(img2, cmap='gray') # useful for grayscale images to avoid false coloring
  else:
    plt.imshow(img2)
  plt.axis('off')

  plt.subplot(1, 3, 3)
  plt.title(title3)
  if isGrayscale(img3):
    plt.imshow(img3, cmap='gray') # useful for grayscale images to avoid false coloring
  else:
    plt.imshow(img3)
  plt.axis('off')

  plt.show()


# Task


# Theory

* Kameras kalibrēšana ir process, kas nosaka, kā kamera pārveido 3D
pasauli 2D attēlā.
* Bez kalibrēšanas attēlā:
  - līnijas var izskatīties izliektas (optiskais kropļojums),
  - objektu izmērus un attālumus nevar precīzi noteikt,
  - stereo pāros nevar korekti aprēķināt dziļumu.
* Kalibrēšana ļauj:
  - novērst objektīva kropļojumus,
  - noteikt fokusa garumu un galveno punktu,
  - aprēķināt reālos mērogus un attālumus no attēla,
  - sagatavot kameru stereo redzei un 3D rekonstrukcijai.

Kalibrēšanas rezultāts ir divu tipu parametri:
 - Iekšējie (intrinsic), raksturo pašas
kameras optiku un sensoru.
    - fokusa garumi (pikseļos)
    - attēla centrs
    - kropļojuma koeficienti
 - Ārējie (extrinsic), raksturo kameras
pozīciju un orientāciju telpā.
    - 𝑅: rotācijas matrica
    - 𝑡: translācijas vektors

Reālām lēcām ir:
 - Radiālais kropļojums: 𝑘1 , 𝑘2 , 𝑘3
 - Tangenciālais kropļojums: 𝑝1 , 𝑝2
 - Modelis: 𝑥𝑑𝑖𝑠𝑡 = 𝑥 (1 + 𝑘1 𝑟^2 + 𝑘2 𝑟^4 + 𝑘3 𝑟^6) + 2𝑝1 𝑥𝑦
+ 𝑝2 (𝑟^2 + 2𝑥^2)
 - Kalibrācija ļauj šos koeficientus noteikt un attēlus
izlīdzināt (undistort).

Kalibrēšanas praktiskie ieguvumi
- Pareizs ģeometriskais attēls
Noņem izliektas malas, padara taisnas līnijas taisnas.
- Reālistisks dziļuma aprēķins
Disparitātes aprēķins kļūst precīzs, ja zināms fokusa garums un bāzes
attālums.
- Mērījumu uzticamība
Var noteikt attālumus, izmērus, laukumus milimetros vai metros.
- Labāka stereo rektifikācija
Attēli tiek izlīdzināti, lai atbilstības būtu vienā rindā.
- Neironu tīklu ievade
Precīzāki dati ļauj labāk mācīt modeļus (piemēram, dziļuma kartes vai 3D
segmentāciju).

Kalibrācija apraksta kameras īpašības, nevis konkrētu attālumu
- Kameras iekšējie parametri (intrinsics):
  - fokusa garums 𝑓𝑥 , 𝑓𝑦 ,
  - attēla centrs 𝑐𝑥 , 𝑐𝑦 ,
- kropļojuma koeficienti (𝑘1 , 𝑘2 , 𝑝1 , 𝑝2 ) ir nemainīgi, kamēr:
  - tu nemaini fokusa attālumu (zoom),
  - tu nepārvietojies uz citu fokusa režīmu (piem., auto → macro),
  - tu neizmaini izšķirtspēju vai sensors nav kustīgs.
- Tātad kalibrāciju vari izmantot visiem attālumiem — tā “izlabo” lēcas
un sensoru ģeometriju.

Kas jāievēro pie kalibrācijas
- Kalibrācijas dati kļūst mazāk precīzi, ja:
  - šablons tika fotografēts tikai ļoti tuvu kamerai, bet objekti pēc tam ir tālu (vai
otrādi);
  - šablons bija tikai vienā leņķī, bet lietojumā objekti ir citos skatījumos;
  - objektīvs ir ar mainīgu fokusa garumu (zoom) un tas mainās starp
uzņemšanām;
  - kamera ir mobilā ierīce ar maināmu autofokusu — fokusa garums var
mainīties automātiski.

**Ieteikumi
precīzai
kalibrēšanai**
- Fotografē šaha dēli dažādos attālumos un
leņķos, lai modelis būtu stabils.
- Ja paredzēts darbs aptuveni vienā diapazonā
(piem., 0.5–2 m), tad kalibrē arī apmēram šajā
diapazonā.
- Ja maini zoom vai fokusa režīmu – jākalibrē
no jauna.
- Ja strādā ar viedtālruņiem, centies fiksēt
fokusu (lock focus) pirms attēlu uzņemšanas.

**Tipiskās
kļūdas**
- Neprecīza kalibrācija (maz attēlu, slikts leņķu
sadalījums)
- Attēlu rotācija vai novirze starp kadriem
- Gaismas un ekspozīcijas atšķirības
- Vienmuļas virsmas bez tekstūras
- Pārāk maza bāze (mazs BBB)

Kā atrast fokusa garumu?
(Zhanga metode)
 - Fotografē zināmu šablonu (piem., šaha laukumu)
no dažādiem leņķiem.
 - Atrod stūra punktus katrā attēlā.
 -  Aprēķina projekcijas matricas un iegūst 𝐾, 𝑅, 𝑡.
 -  Optimizē, samazinot projicēšanas kļūdu 𝑒

- Katrs šaha dēļa stūra punkts ir zināms reālajā pasaulē (X, Y, 0).
- Kad mēs šos punktus redzam dažādās attēla pozīcijās, mēs iegūstam
daudzas vienādojumu sistēmas
- Ar vienu attēlu mēs nevaram unikāli noteikt visus parametrus (pārāk
maz datu). Tāpēc:
- uzņemam vairākus attēlus,
- katrs attēls dod jaunus vienādojumus,
- tiek atrisināts kopīgs optimizācijas uzdevums.

Kā tiek atrasts fokusa garums
- Zināmi pasaules punkti (šaha dēļa koordinātas)
- Zināmi to pikseļu koordinātas attēlā
- Nezināmie: 𝑓𝑥 , 𝑓𝑦 , 𝑐𝑥 , 𝑐𝑦 , 𝑘1 , 𝑘2 , . . .
- Zhang metode atrisina optimizācijas problēmu
- Tiek meklētas tādas 𝑓𝑥 , 𝑓𝑦 ,kas vislabāk atbilst visiem attēliem
vienlaikus, samazinot projicēšanas kļūdu starp reālo un aprēķināto
stūri.

###3. Disparitātes un dziļuma karte ar reāliem attēliem
* Ar savu viedtālruni uzņemt vismaz 10 foto ar šaha laukuma šablonu (no dažādiem leņķiem un attālumiem).
* Izmantot cv.findChessboardCorners() un cv.calibrateCamera() lai:
noteiktu kameras matricu 𝐾un kropļojuma koeficientus 𝑑𝑖𝑠𝑡,
saglabā rezultātu camera_calib_params.npz.
* Fotografēt divus attēlus (viena aina, bet kamera nobīdīta horizontāli par zināmu bāzi 𝐵, piemēram, 6 cm).
* Izveidot:
  * Disparitātes karti ar StereoSGBM
  * Dziļuma karti, izmantojot 𝑓no kalibrācijas un 𝐵 no mērījuma
* Vizualizēt un kvalitatīvi salīdzināt rezultātus ar Middlebury datiem.


In [375]:
"""
  Kalibrācija – cv.findChessboardCorners, cv.calibrateCamera()
  Attēlu izlīdzināšana (undistort) – cv.undistort()
  Rektifikācija – cv.stereoRectify()

  https://forum.opencv.org/t/asymmetric-circle-grid-calibration/21121/8
  https://longervision.github.io/2017/03/18/ComputerVision/OpenCV/opencv-internal-calibration-circle-grid/
"""

'\n  Kalibrācija – cv.findChessboardCorners, cv.calibrateCamera()\n  Attēlu izlīdzināšana (undistort) – cv.undistort()\n  Rektifikācija – cv.stereoRectify()\n\n  https://forum.opencv.org/t/asymmetric-circle-grid-calibration/21121/8\n  https://longervision.github.io/2017/03/18/ComputerVision/OpenCV/opencv-internal-calibration-circle-grid/\n'

## old version calib data used (max0 phone)
Found 9 images

=== Calibration Result ===
Reprojection error: 82.58017793478639

Camera matrix:
 [[ 4.32096944e+03  0.00000000e+00  2.32543921e+03]
 [ 0.00000000e+00  2.94401714e+03 -3.28903232e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Camera matrix f_px:
 4320.969436320265 <-- spoofed

Distortion coefficients:
 [-0.22217271 -0.49032087  0.19050592  0.01774387  0.2797825 ]

===========================

# version 2

In [376]:
# --- Calibration & Stereo Pipeline Configuration ---
PATT_SIZE = (4,11)

# Toggleable options for A/B testing
USE_FIXED_BASELINE = True          # If True, use BASELINE_MM for metric depth; else use recovered scale
GRID_BASELINE_MM = 47.0                 # nominal baseline in mm (for object stereo) distance between circles in grid
GRID_BASELINE_M = GRID_BASELINE_MM / 1000

WORLD_ORIGIN = 'pattern'           # 'pattern' (OpenCV default) or 'first_camera'
RECTIFICATION_MODE = 'stereoRectify'  # options: 'stereoRectify', 'homography', 'affine'

USE_XIMGPROC_WLS = True
USE_EXIF_INTRINSICS = True
EXIF_TO_USE = ['FocalLength','FocalLengthIn35mmFormat','Make','Model','ImageWidth','ImageHeight']

ASSUME_PURE_HORIZONTAL_SHIFT = True
ALLOW_WOBBLE_TOLERANCE_DEG = 2.0

# Bilateral / SGBM presets (tune these)
BILATERAL_FILTER = True
BILATERAL_PARAMS = {'d':9, 'sigmaColor':75, 'sigmaSpace':75}
SGBM_PARAMS = {'minDisparity':0, 'numDisparities':128, 'blockSize':3}

# EXIF helpers (exiftool preferred, fallback to Pillow+piexif)
import subprocess, shlex
def read_exif_with_exiftool(img_path):
    try:
        cmd = f"exiftool -j -FocalLength -FocalLengthIn35mmFormat -Make -Model -ImageWidth -ImageHeight -LensModel {shlex.quote(img_path)}"
        p = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if p.returncode != 0:
            return {}
        import json as _json
        arr = _json.loads(p.stdout)
        return arr[0] if arr else {}
    except Exception as e:
        print('exiftool error:', e)
        return {}

def read_exif_pillow(img_path):
    out = {}
    try:
        from PIL import Image
        import piexif
        img = Image.open(img_path)
        exif = img.getexif()
        orientation = exif.get(274)  # 274 is orientation tag
        print(f"EXIF Orientation: {orientation}")
        # 1 = normal, 6 = rotated 90° CW, 8 = rotated 90° CCW, 3 = 180°
        exif_bytes = img.info.get('exif', None)
        if exif_bytes:
            exif_dict = piexif.load(exif_bytes)
            fl = exif_dict['Exif'].get(piexif.ExifIFD.FocalLength, None)
            if fl:
                out['FocalLength'] = float(fl[0]) / float(fl[1])
            w, h = img.size
            out['ImageWidth']  = w
            out['ImageHeight'] = h
            """
             "EncodingProcess": "Baseline DCT, Huffman coding",
              "ColorComponents": 3,
              "Megapixels": 12.0,
              "LightValue": 12.7
            """
            print(exif_dict)
            # --- 0th IFD ---
            zeroth = exif_dict.get('0th', {})

            make = zeroth.get(piexif.ImageIFD.Make)
            if make is not None:                out['Make'] = make.decode() if isinstance(make, bytes) else make

            model = zeroth.get(piexif.ImageIFD.Model)
            if model is not None:                out['Model'] = model.decode() if isinstance(model, bytes) else model

            # yota       "ImageWidth": 3000,              "ImageHeight": 4000,
            width = zeroth.get(piexif.ImageIFD.ImageWidth)
            height = zeroth.get(piexif.ImageIFD.ImageHeight)
            if width is not None:                out['ImageWidth'] = width
            if height is not None:                out['ImageHeight'] = height

            bits_per_sample = zeroth.get(piexif.ImageIFD.BitsPerSample) # "BitsPerSample": 8,
            if bits_per_sample is not None:                out['BitsPerSample'] = bits_per_sample

            ycbcr_subsampling = zeroth.get(piexif.ImageIFD.YCbCrSubSampling) # "YCbCrSubSampling": "YCbCr4:2:0 (2 2)",
            if ycbcr_subsampling is not None:
                out['YCbCrSubSampling'] = ycbcr_subsampling

            ycbcr_positioning = zeroth.get(piexif.ImageIFD.YCbCrPositioning) #  "YCbCrPositioning": "Centered",
            if ycbcr_positioning is not None:                out['YCbCrPositioning'] = ycbcr_positioning

            # --- Exif IFD ---
            exif_ifd = exif_dict.get('Exif', {})

            exposure_time = exif_ifd.get(piexif.ImageIFD.ExposureTime) #"ExposureTime": "1/1809",
            if exposure_time is not None:
                out['ExposureTime'] = exposure_time
                out['ShutterSpeed'] = exposure_time    #same value
                out['ShutterSpeedValue'] = exposure_time #same value

            iso = exif_ifd.get(piexif.ExifIFD.ISOSpeedRatings) #"ISO": 100,
            if iso is not None:        out['ISO'] = iso

            fnumber = exif_ifd.get(piexif.ExifIFD.FNumber) #"FNumber": 1.9,
            if fnumber is not None:
                out['FNumber'] = fnumber
                out['Aperture'] = fnumber  # same physical value
                out['ApertureValue'] = fnumber  # same physical value

            brightness = exif_ifd.get(piexif.ExifIFD.BrightnessValue) # "BrightnessValue": 0,
            if brightness is not None:        out['BrightnessValue'] = brightness

            metering = exif_ifd.get(piexif.ExifIFD.MeteringMode) # "MeteringMode": "Unknown",
            if metering is not None:        out['MeteringMode'] = metering

            flash = exif_ifd.get(piexif.ExifIFD.Flash) # "Flash": "Off, Did not fire",
            if flash is not None:        out['Flash'] = flash

            focal_length = exif_ifd.get(piexif.ExifIFD.FocalLength) # "FocalLength": "3.9 mm",
            if focal_length is not None:
                out['FocalLength'] = focal_length
                out['FocalLength35efl'] = focal_length #"FocalLength35efl": "3.9 mm",

            # --- ColorComponents: inferred only if BitsPerSample is a tuple ---
            if bits_per_sample is not None and isinstance(bits_per_sample, tuple):        out['ColorComponents'] = len(bits_per_sample)

    except Exception:
        pass
    return out

def exif_to_initial_fxfy(exif, img_width_px):
    # Estimate sensor width from typical smartphone range if unknown (approx 4.8 mm)
    f_mm = None
    if 'FocalLength' in exif:
        try:
            f_mm = float(str(exif['FocalLength']).split()[0])
        except Exception:
            pass
    if f_mm is None and 'FocalLengthIn35mmFormat' in exif:
        try:
            f35 = float(str(exif['FocalLengthIn35mmFormat']).split()[0])
            f_mm = 4.8 * (f35 / 43.3)
        except Exception:
            pass
    if f_mm is None:
        return None, None
    sensor_width_mm = 4.8
    fx_px = (f_mm / sensor_width_mm) * img_width_px
    fy_px = fx_px
    return fx_px, fy_px

# Note: This notebook/patch assumes rear camera (YOTA 3+) used for both calibration and stereo processing.

In [377]:
!pip install opencv-contrib-python -U -q

import cv2
import numpy as np
import glob
import yaml
import os

from typing import List, Tuple, Dict
from collections import namedtuple

from cv2 import ximgproc
#from cv2.ximgproc import createDisparityWLSFilterGeneric # out of date

In [405]:
"""
Patch helper module for modifying your existing notebooks:
- Adds improved asymmetric-circles calibration routine
- Adds expanded YAML exporter with required fields
- Adds motion-stereo baseline estimation (recoverPose) and stereo->PLY pipeline
- Uses OpenCV (opencv-python, opencv-contrib-python for ximgproc), numpy, PyYAML

Usage: import this module from your notebook or run functions directly.
"""

# ------------------------- Calibration helpers -------------------------
ImageData = namedtuple('ImageData', ['path', 'img'])
from enum import Enum

class Orientation:
    Landscape = "landscape"
    Portrait = "portrait"

def load_gray_images(folder: str, pattern: str = "*.jpg", fix_orientation=Orientation.Landscape) -> List[ImageData]:
    """Load grayscale images and return ImageData objects."""
    files = sorted(glob.glob(os.path.join(folder, pattern)))
    result = []

    for f in files:
        im = cv2.imread(f)
        if im is None:
            print(f"Warning: Could not load {f}")
            continue


        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        if fix_orientation is not None:
            h, w = gray.shape
            if h > w and fix_orientation == Orientation.Landscape:
                print(f"  correcting shape {im.shape} to {fix_orientation} for {f}")
                gray = cv2.rotate(gray, cv2.ROTATE_90_CLOCKWISE)
            elif h < w and fix_orientation == Orientation.Portrait:
                print(f"  correcting shape {im.shape} to {fix_orientation} for {f}")
                gray = cv2.rotate(gray, cv2.ROTATE_90_COUNTERCLOCKWISE)

        print(f"OpenCV shape(height, width, channels): {gray.shape}")

        result.append(ImageData(path=f, img=gray))

    print(f"Loaded {len(result)} images")
    return result

def detect_asymmetric_circles(
    images: List[ImageData],
    pattern_size: Tuple[int,int] = (11, 4),
    pattern_spacing_m: float = GRID_BASELINE_M,
    blobParams = cv2.SimpleBlobDetector_Params(),
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # criteria to stop refining; todo how to tune?
) -> Dict:
    # build object points for asymmetric circles grid (OpenCV asymmetric layout)
    cols, rows = pattern_size
    objp = []
    for y in range(rows):
        for x in range(cols):
            # asymmetric grid: x positions offset every other row
            objp.append([ (2*x + (y % 2)) * (pattern_spacing_m/2.0), y * pattern_spacing_m, 0.0 ])
    objp = np.array(objp, dtype=np.float32)

    found_count = 0

    def process_pattern(gray, centers, found, pattern_size, objp, criteria):
      # Draw original corners
      #im_chess = cv2.drawChessboardCorners(gray, pattern_size, centers, found)

      # Refine corners
      corners_refined = cv2.cornerSubPix(gray, centers,
                                          (11,11), # Window size: search in 11×11 pixel area around each point
                                           (-1,-1), # Dead zone: -1 means no dead zone (use full window)
                                         criteria)
      #im_chess_refi = cv2.drawChessboardCorners(gray, pattern_size, corners_refined, found)

      # Draw connecting lines
      #img_color = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
      #for i in range(len(corners_refined) - 1):
      #    pt1 = tuple(corners_refined[i].ravel().astype(int))
      #    pt2 = tuple(corners_refined[i+1].ravel().astype(int))
      #    cv2.line(img_color, pt1, pt2, (255, 0, 0), 2)  # thickness=2

      #plotImgSideBySide(im_chess, "corners", img_color, 'refined')

      return corners_refined

    # Create a detector with the parameters
    blobDetector = cv2.SimpleBlobDetector_create(blobParams)

    blob_data = []
    for idx, imgdata in enumerate(images):
        gray = imgdata.img.copy()

        if True: # for debug only
          """
            findCirclesGrid() detects blobs internally, so Drawing keypoints on the image doesn't help the algorithm.
            The colored circles get converted to gray, which adds noise rather than helping.
          """
          keypoints = blobDetector.detect(gray) # Detect blobs.
          print(f"[{idx}]: Detected {len(keypoints)} blobs for {imgdata.path}")

        found, centers = cv2.findCirclesGrid(gray, pattern_size, flags=cv2.CALIB_CB_ASYMMETRIC_GRID, blobDetector=blobDetector)
        if found:
            found_count += 1
            print(f"[{idx}]: found asymmetric pattern {found_count} out of {len(images)}")
            centers = process_pattern(gray, centers, found, pattern_size, objp, criteria)

        else:
            # try with additional flags if failed: adaptive threshold + normalize
            found, centers2 = cv2.findCirclesGrid(gray, pattern_size,
                                                  flags=cv2.CALIB_CB_ASYMMETRIC_GRID + cv2.CALIB_CB_CLUSTERING, blobDetector=blobDetector)
            if found:
                found_count += 1
                print(f"[{idx}]: found asymmetric pattern with clustering {found_count} out of {len(images)}")
                centers = process_pattern(gray, centers2, found, pattern_size, objp, criteria)

        if found:
            blob_data.append(
                {
                  'path': imgdata.path,
                  'objp': objp,
                  'centers': centers,
                  'idx':idx
                }
            )
    if len(blob_data) < 6:
        raise ValueError(f"Not enough good detections ({len(blob_data)}). Need >=6 images spanning pose variations.")

    print(f"Found {found_count} out of {len(images)} images")

    return blob_data

def save_calibration_yaml(metadata: Dict, filename: str):
    with open(filename, 'w') as f:
        yaml.safe_dump(metadata, f, sort_keys=False)
    print(f"Saved calibration YAML to {filename}")

def save_calibration_npz(metadata: Dict, filename: str):
    np.savez(filename, **metadata)
    """
    (f'{calib_file}.npz',
         cam0=mtx,
         baseline=0.1,
         distortion_coefficients=dist,
         rvecs=rvecs,
         tvecs=tvecs)
    """

def save_calibration_json(metadata: Dict, filename: str):
    with open(filename, 'w') as f:
        json.dump(metadata, f, sort_keys=False, indent=4)
    print(f"Saved calibration JSON to {filename}")


"""Calibrate using asymmetric circles grid.

pattern_size: (cols, rows) count of circle centers (OpenCV order)
pattern_spacing_m: center-to-center spacing in meters

Returns raw K, dist, rvecs, tvecs, reprojection error
"""
def calibrate_camera(blob_data, img_shape, flags: int = cv2.CALIB_ZERO_TANGENT_DIST):
    objpoints = [d['objp'] for d in blob_data]
    imgpoints = [d['centers'] for d in blob_data]

    print(f" > img shape: {img_shape}")
    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None, flags=flags)
    for i, (rvec, tvec) in enumerate(zip(rvecs, tvecs)):
      blob_data[i]['rvec'] = rvec
      blob_data[i]['tvec'] = tvec
    print(f"Initial calibration RMS: {ret:.3f}")

    return blob_data, ret, K, dist

def projectPointsErrors(blob_data, mtx, dist):
    errors = []
    for i, data in enumerate(blob_data):
        idx, objp, imgp, rvec, tvec = itemgetter('idx', 'objp', 'centers', 'rvec', 'tvec')(data)

        error = projectPointsError(objp, imgp, rvec, tvec, mtx, dist)
        errors.append(error)
        blob_data[i]['error'] = error
        print(f"  Image {idx} {blob_data[i]['path']}: error = {error:.3f}")

    return errors

def projectPointsError(objp, imgp, rvec, tvec, mtx, dist):
    imgpoints2, _ = cv2.projectPoints(objp, rvec, tvec, mtx, dist)
    error = cv2.norm(imgp, imgpoints2, cv2.NORM_L2) / len(imgpoints2)

    return error

def printProjectPointsErrorStats(blob_data, errors=None):
    errors = [d['error'] for d in blob_data] if errors is None else errors

    print(f"\nOverall projectPoint statistics for {len(errors)} samples:")
    print(f"  Min err: {min(errors):.0f}")
    print(f"  Max err: {max(errors):.0f}")
    print(f"  Median err: {sorted(errors)[len(errors)//2]:.0f}")
    print(f"  Mean err: {sum(errors)/len(errors):.0f}")

    # Check distribution
    errors = np.array(errors)

    print(f"25th percentile: {np.percentile(errors, 25):.1f}")
    print(f"50th percentile: {np.percentile(errors, 50):.1f}")
    print(f"75th percentile: {np.percentile(errors, 75):.1f}")
    print(f"90th percentile: {np.percentile(errors, 90):.1f}")

from operator import itemgetter
"""
  Finetune prepared calibration data
"""
def calibrate_with_outlier_removal(blob_data, ret, mtx, dist, img_shape, threshold=0, flags: int = cv2.CALIB_ZERO_TANGENT_DIST):
    """
    Calibrate, then remove high-error images and recalibrate
    """
    print(f" > img shape: {img_shape}")
    # Calculate per-image errors
    errors = projectPointsErrors(blob_data, mtx, dist)
    printProjectPointsErrorStats(None, errors)


    if threshold == 0: # adaptive mode
        threshold = sorted(errors)[len(errors)//2]
        print(f"\nUsing mean error as threshold: {threshold}")

    # Filter out high-error images
    good_indices = [i for i, err in enumerate(errors) if err < threshold]

    good_blobs = [d for d in blob_data if d['error'] <= threshold]

    if len(good_blobs) < len(blob_data):
        print(f"\nRemoving {len(blob_data) - len(good_blobs)} high-error images")

        objpoints_filtered = [d['objp'] for d in good_blobs] #[objpoints[i] for i in good_indices]
        imgpoints_filtered = [d['centers'] for d in good_blobs] #[imgpoints[i] for i in good_indices]

        # Second pass: recalibrate without outliers
        ret2, mtx2, dist2, rvecs2, tvecs2 = cv2.calibrateCamera(
            objpoints_filtered, imgpoints_filtered, img_shape, None, None, flags=flags
        )

        for i, (rvec, tvec) in enumerate(zip(rvecs2, tvecs2)):
          good_blobs[i]['rvec'] = rvec
          good_blobs[i]['tvec'] = tvec
          good_blobs[i]['error'] = projectPointsError(objpoints_filtered[i], imgpoints_filtered[i], rvec, tvec, mtx2, dist2)

        print(f"Refined calibration RMS: {ret2:.3f}")
        print(f"Using {len(good_indices)}/{len(blob_data)} images")

        return good_blobs, ret2, mtx2, dist2 #, used_imgs_filtered
    else:
        print("\nAll images passed error threshold")
        return good_blobs, ret, mtx, dist #, used_imgs

In [510]:
imgs = load_gray_images(folder_path_root,
                        #fix_orientation=Orientation.Landscape # rms 33.952, 64 pics; distance: 1.8 - 6.4m
                                                              # with scale outliers removed:
                                                              #   rms 32.674, 62 pics, distance: 1.8 - 6.4
                                                              #    x_fpx ~40k y_fpx ~22k
                        fix_orientation=Orientation.Portrait  # rms 33.502, 62 pics, dist: 1.9 - 7m fpx ~70k
                                                              # with scale outliers removed:
                                                              #   rms 32.595,  58 pics, distance: 1.8 - 6.8,
                                                              #    x_fpx ~23k y_fpx ~40k
                        )

OpenCV shape(height, width, channels): (4000, 3000)
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131111.jpg
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131116.jpg
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131118.jpg
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131121.jpg
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131125.jpg
OpenCV shape(height, width, channels): (4000, 3000)
  correcting shape (3000, 4000, 3) to portrait for calibrate2/IMG_20251107_131127.jpg
OpenCV shape(height, width, channels): (4000, 3000)
OpenCV shape(height, width, channels): (4000, 3000)
OpenCV shape(hei

In [402]:
import math
# Check blob sizes across all images
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = True
params.minArea = 5000
params.maxArea = 35000
blobDetector = cv2.SimpleBlobDetector_create(params)

all_areas = []
counter = 0
for idx, imgdata in enumerate(imgs):
    print(f"[{idx}]: Checking blob sizes {imgdata.path}")
    gray = imgdata.img
    keypoints = blobDetector.detect(gray)
    if keypoints:
        print(f"[{idx}]: Detected {len(keypoints)} blobs")

        if len(keypoints) < 44:
            print(f"[{idx}]: Not enough blobs, skipping")
            continue

        counter += 1

        # Get both size (diameter) and calculated area
        sizes = [kp.size for kp in keypoints]
        areas = [math.pi * (kp.size/2)**2 for kp in keypoints]  # Convert diameter to area

        all_areas.extend(areas)

        print(f"Image {idx}: blob areas {min(areas):.0f} - {max(areas):.0f}")

if all_areas:
    print(f" {counter} pictures out of {len(imgs)} have enough blobs: ")
    print(f"\nOverall statistics:")
    print(f"  Min area: {min(all_areas):.0f}")
    print(f"  Max area: {max(all_areas):.0f}")
    print(f"  Median area: {sorted(all_areas)[len(all_areas)//2]:.0f}")
    print(f"  Mean area: {sum(all_areas)/len(all_areas):.0f}")

    # Suggested parameters (covering 95% of blobs)
    all_areas.sort()
    min_suggested = all_areas[int(len(all_areas) * 0.05)]
    max_suggested = all_areas[int(len(all_areas) * 0.95)]
    print(f"\nSuggested parameters (covering 95% of blobs):")
    print(f"  minArea: {min_suggested:.0f}")
    print(f"  maxArea: {max_suggested:.0f}")

[0]: Checking blob sizes calibrate2/IMG_20251107_131104.jpg
[0]: Detected 44 blobs
Image 0: blob areas 6339 - 10256
[1]: Checking blob sizes calibrate2/IMG_20251107_131107.jpg
[1]: Detected 44 blobs
Image 1: blob areas 9309 - 20985
[2]: Checking blob sizes calibrate2/IMG_20251107_131111.jpg
[2]: Detected 44 blobs
Image 2: blob areas 11216 - 30431
[3]: Checking blob sizes calibrate2/IMG_20251107_131116.jpg
[3]: Detected 44 blobs
Image 3: blob areas 8040 - 17708
[4]: Checking blob sizes calibrate2/IMG_20251107_131118.jpg
[4]: Detected 44 blobs
Image 4: blob areas 11706 - 25583
[5]: Checking blob sizes calibrate2/IMG_20251107_131121.jpg
[5]: Detected 44 blobs
Image 5: blob areas 5785 - 9951
[6]: Checking blob sizes calibrate2/IMG_20251107_131125.jpg
[6]: Detected 44 blobs
Image 6: blob areas 10488 - 27940
[7]: Checking blob sizes calibrate2/IMG_20251107_131127.jpg
[7]: Detected 44 blobs
Image 7: blob areas 8769 - 22363
[8]: Checking blob sizes calibrate2/IMG_20251107_131133.jpg
[8]: Detec

In [511]:
########################################Blob Detector##############################################

# Setup SimpleBlobDetector parameters.
blobParams = cv2.SimpleBlobDetector_Params()

#blobParams.minDistBetweenBlobs = 5;

# Change thresholds
#blobParams.minThreshold = 10
#blobParams.maxThreshold = 2000

blobParams.filterByColor = True
blobParams.blobColor = 0

# Filter by Area.
blobParams.filterByArea = True
blobParams.minArea = 20 #64     # filter out all the blobs that have less then xx pixels.
blobParams.maxArea = 35000   # 10k - 8 prj_err ~2.; 15k - 17; 18k - 29; 35k - 79;

# Filter by Circularity
#blobParams.filterByCircularity = True
#blobParams.minCircularity = 0.001

# Filter by Convexity (adds noise)
#blobParams.filterByConvexity = True
#blobParams.minConvexity = 0.17

# Filter by Inertia
#blobParams.filterByInertia = True
#blobParams.minInertiaRatio = 0.01

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

###################################################################################################
#imgs_copy = [img.copy() for img in imgs]
blob_data = detect_asymmetric_circles(imgs, pattern_size=PATT_SIZE,
                                    pattern_spacing_m=GRID_BASELINE_M, #was o.o3
                                    blobParams=blobParams, criteria=criteria)


[0]: Detected 44 blobs for calibrate2/IMG_20251107_131104.jpg
[0]: found asymmetric pattern with clustering 1 out of 81
[1]: Detected 44 blobs for calibrate2/IMG_20251107_131107.jpg
[1]: found asymmetric pattern with clustering 2 out of 81
[2]: Detected 44 blobs for calibrate2/IMG_20251107_131111.jpg
[2]: found asymmetric pattern with clustering 3 out of 81
[3]: Detected 44 blobs for calibrate2/IMG_20251107_131116.jpg
[3]: found asymmetric pattern with clustering 4 out of 81
[4]: Detected 44 blobs for calibrate2/IMG_20251107_131118.jpg
[4]: found asymmetric pattern with clustering 5 out of 81
[5]: Detected 44 blobs for calibrate2/IMG_20251107_131121.jpg
[5]: found asymmetric pattern 6 out of 81
[6]: Detected 44 blobs for calibrate2/IMG_20251107_131125.jpg
[6]: found asymmetric pattern with clustering 7 out of 81
[7]: Detected 44 blobs for calibrate2/IMG_20251107_131127.jpg
[7]: found asymmetric pattern with clustering 8 out of 81
[8]: Detected 44 blobs for calibrate2/IMG_20251107_13113

In [512]:
## OpenCV API expects order (width, height), which is the opposite of NumPy's array shape (height, width)!
## i.e. OpenCV returns data as NumPy arrays, but expects to get shape param reversed
img_shape_cv = imgs[0].img.shape[::-1] #reverse
print(f"img shape: {imgs[0].img.shape}")
blob_data, ret, K, dist = calibrate_camera(blob_data, img_shape_cv)

import copy
blob_data_cp = copy.deepcopy(blob_data)

_ = projectPointsErrors(blob_data_cp, K, dist)


img shape: (4000, 3000)
 > img shape: (3000, 4000)
Initial calibration RMS: 40.693
  Image 0 calibrate2/IMG_20251107_131104.jpg: error = 3.902
  Image 1 calibrate2/IMG_20251107_131107.jpg: error = 6.795
  Image 2 calibrate2/IMG_20251107_131111.jpg: error = 7.950
  Image 3 calibrate2/IMG_20251107_131116.jpg: error = 2.349
  Image 4 calibrate2/IMG_20251107_131118.jpg: error = 4.982
  Image 5 calibrate2/IMG_20251107_131121.jpg: error = 1.708
  Image 6 calibrate2/IMG_20251107_131125.jpg: error = 6.213
  Image 7 calibrate2/IMG_20251107_131127.jpg: error = 5.283
  Image 8 calibrate2/IMG_20251107_131133.jpg: error = 8.554
  Image 9 calibrate2/IMG_20251107_131136.jpg: error = 7.222
  Image 10 calibrate2/IMG_20251107_131139.jpg: error = 7.486
  Image 11 calibrate2/IMG_20251107_131141.jpg: error = 3.929
  Image 12 calibrate2/IMG_20251107_131144.jpg: error = 5.305
  Image 13 calibrate2/IMG_20251107_131147.jpg: error = 5.083
  Image 16 calibrate2/IMG_20251107_131156.jpg: error = 10.391
  Image 17 

In [513]:
good_blobs = copy.deepcopy(blob_data_cp)
ret_cp = float(ret)
K_cp = K.copy()
dist_cp = dist.copy()

printProjectPointsErrorStats(good_blobs)
print(f"Initial calibration RMS: {ret_cp:.3f}")


Overall projectPoint statistics for 72 samples:
  Min err: 2
  Max err: 14
  Median err: 6
  Mean err: 6
25th percentile: 3.9
50th percentile: 5.6
75th percentile: 7.2
90th percentile: 8.4
Initial calibration RMS: 40.693


In [516]:
# After collecting all points recalibrate to minimize error
good_blobs, ret_cp, K_cp, dist_cp = calibrate_with_outlier_removal(
    good_blobs, ret_cp, K_cp, dist_cp, img_shape_cv,
    #threshold=0 # zero for auto
    threshold=8 # more is better for coverage, so don't remove too much; use 90th percentile
)

 > img shape: (3000, 4000)
  Image 0 calibrate2/IMG_20251107_131104.jpg: error = 3.834
  Image 1 calibrate2/IMG_20251107_131107.jpg: error = 6.498
  Image 2 calibrate2/IMG_20251107_131111.jpg: error = 10.584
  Image 3 calibrate2/IMG_20251107_131116.jpg: error = 2.831
  Image 4 calibrate2/IMG_20251107_131118.jpg: error = 3.415
  Image 5 calibrate2/IMG_20251107_131121.jpg: error = 1.277
  Image 6 calibrate2/IMG_20251107_131125.jpg: error = 4.621
  Image 7 calibrate2/IMG_20251107_131127.jpg: error = 4.011
  Image 9 calibrate2/IMG_20251107_131136.jpg: error = 6.807
  Image 10 calibrate2/IMG_20251107_131139.jpg: error = 6.296
  Image 11 calibrate2/IMG_20251107_131141.jpg: error = 3.089
  Image 12 calibrate2/IMG_20251107_131144.jpg: error = 4.528
  Image 13 calibrate2/IMG_20251107_131147.jpg: error = 7.103
  Image 18 calibrate2/IMG_20251107_131202.jpg: error = 9.169
  Image 19 calibrate2/IMG_20251107_131207.jpg: error = 6.734
  Image 20 calibrate2/IMG_20251107_131210.jpg: error = 6.197
  Ima

In [518]:
_ = projectPointsErrors(good_blobs, K_cp, dist_cp)
printProjectPointsErrorStats(good_blobs)
print(f"Tuned calibration RMS: {ret_cp:.3f}")

  Image 0 calibrate2/IMG_20251107_131104.jpg: error = 3.492
  Image 1 calibrate2/IMG_20251107_131107.jpg: error = 6.119
  Image 3 calibrate2/IMG_20251107_131116.jpg: error = 2.827
  Image 4 calibrate2/IMG_20251107_131118.jpg: error = 3.415
  Image 5 calibrate2/IMG_20251107_131121.jpg: error = 1.405
  Image 6 calibrate2/IMG_20251107_131125.jpg: error = 4.381
  Image 7 calibrate2/IMG_20251107_131127.jpg: error = 3.816
  Image 9 calibrate2/IMG_20251107_131136.jpg: error = 6.272
  Image 10 calibrate2/IMG_20251107_131139.jpg: error = 6.159
  Image 11 calibrate2/IMG_20251107_131141.jpg: error = 3.112
  Image 12 calibrate2/IMG_20251107_131144.jpg: error = 4.509
  Image 13 calibrate2/IMG_20251107_131147.jpg: error = 6.196
  Image 19 calibrate2/IMG_20251107_131207.jpg: error = 6.776
  Image 20 calibrate2/IMG_20251107_131210.jpg: error = 6.170
  Image 21 calibrate2/IMG_20251107_131212.jpg: error = 4.348
  Image 22 calibrate2/IMG_20251107_131215.jpg: error = 1.773
  Image 23 calibrate2/IMG_202511

##Before vs After Comparison
BEFORE: 84 images, errors 1-13, mean 5, RMS: 36.452

AFTER:  71 images, errors 1-8,  mean 5, RMS: 33.452

---
**Results**:
 - ✅ Removed 13 worst images (top 15%)
 - ✅ RMS improved: 36.452 → 33.452 (8% reduction)
 - ✅ Max error: 13 → 8 (removed outliers)
 - ✅ Distribution tightened
 - ✅ Still have 71 images (good coverage)

In [503]:
def prepare_calibration_metadata(blob_data, ret, K, dist, img_shape_cv, pattern_size=PATT_SIZE, pattern_spacing_m=GRID_BASELINE_M):
    cols, rows = pattern_size
    rvecs = [d['rvec'] for d in blob_data]
    tvecs = [d['tvec'] for d in blob_data]
    used_imgs = [d['path'] for d in blob_data]
    w,h = img_shape_cv

    distances = np.linalg.norm(tvecs, axis=1)

    print(f"Distance statistics (meters):")
    print(f"  Min: {np.min(distances):.2f}m")
    print(f"  Max: {np.max(distances):.2f}m")
    print(f"  Mean: {np.mean(distances):.2f}m")
    print(f"  Median: {np.median(distances):.2f}m")

    # prepare expanded metadata
    metadata = {
        'pattern_type': 'asymmetric_circles',
        'pattern_size': {'cols': cols, 'rows': rows},
        'pattern_spacing_m': float(pattern_spacing_m),
        'image_resolution': [int(w), int(h)],
        'reprojection_error': float(ret),
        'camera_matrix': K.tolist(),
        'cam0': K.tolist(),
        'fx': float(K[0,0]),
        'fy': float(K[1,1]),
        'cx': float(K[0,2]),
        'cy': float(K[1,2]),
        'distortion_model': 'opencv_radial_tangential',
        'distortion_coeffs_count': int(dist.size),
        'distortion_coefficients': dist.ravel().tolist(),
        'rvecs_count': len(rvecs),
        'tvecs_count': len(tvecs),
        'image_indices_used': used_imgs
    }

    # include rvecs/tvecs as arrays for debugging
    metadata['rvecs'] = [r.reshape(3).tolist() for r in rvecs]
    metadata['tvecs'] = [t.reshape(3).tolist() for t in tvecs]

    return metadata


In [519]:
meta = prepare_calibration_metadata(good_blobs, ret_cp, K_cp, dist_cp, img_shape_cv)

print(meta)
print('\n=== Calibration Result ===')
print(f'Reprojection error: {meta['reprojection_error']}')
print('\nCamera matrix:\n', meta['camera_matrix'])
print('\nCamera matrix f_px x:\n', meta['fx'])
print('\nCamera matrix f_px y:\n', meta['fy'])
print('\nDistortion coefficients:\n', meta['distortion_coefficients'])
print('\n===========================')


Distance statistics (meters):
  Min: 1.88m
  Max: 6.44m
  Mean: 3.36m
  Median: 3.21m
{'pattern_type': 'asymmetric_circles', 'pattern_size': {'cols': 4, 'rows': 11}, 'pattern_spacing_m': 0.047, 'image_resolution': [3000, 4000], 'reprojection_error': 32.59519307365116, 'camera_matrix': [[22148.342864410333, 0.0, 1808.1776756625702], [0.0, 40322.85912032819, 2117.726224634114], [0.0, 0.0, 1.0]], 'cam0': [[22148.342864410333, 0.0, 1808.1776756625702], [0.0, 40322.85912032819, 2117.726224634114], [0.0, 0.0, 1.0]], 'fx': 22148.342864410333, 'fy': 40322.85912032819, 'cx': 1808.1776756625702, 'cy': 2117.726224634114, 'distortion_model': 'opencv_radial_tangential', 'distortion_coeffs_count': 5, 'distortion_coefficients': [-25.44967924596774, -170.1616212145698, 0.0, 0.0, 26.9662069811287], 'rvecs_count': 58, 'tvecs_count': 58, 'image_indices_used': ['calibrate2/IMG_20251107_131104.jpg', 'calibrate2/IMG_20251107_131107.jpg', 'calibrate2/IMG_20251107_131116.jpg', 'calibrate2/IMG_20251107_131118.

In [505]:
read_exif_pillow(good_blobs[0]['path'])

EXIF Orientation: None
{'0th': {271: b'QUALCOMM YotaDevices', 272: b'YOTA 3+', 282: (72, 1), 283: (72, 1), 296: 2, 305: b'Y3-user 8.1.0 OPM1.171019.019 52 dev-keys', 306: b'2025:11:07 13:11:04', 531: 1, 34665: 242, 34853: 726}, 'Exif': {33434: (1, 1809), 33437: (190, 100), 34850: 0, 34855: 100, 36864: b'0220', 36867: b'2025:11:07 13:11:04', 36868: b'2025:11:07 13:11:04', 37121: b'\x01\x02\x03\x00', 37377: (10821, 1000), 37378: (185, 100), 37379: (0, 100), 37383: 0, 37385: 16, 37386: (3940, 1000), 37520: b'044341', 37521: b'044341', 37522: b'044341', 40960: b'0100', 40961: 1, 40962: 3000, 40963: 4000, 40965: 695, 41495: 0, 41729: b'\x00', 41986: 0, 41987: 0, 41989: 0, 41990: 0}, 'GPS': {5: (0, 100)}, 'Interop': {1: b'R98'}, '1st': {}, 'thumbnail': None}


{'FocalLength': 3.94,
 'ImageWidth': 3000,
 'ImageHeight': 4000,
 'Make': 'QUALCOMM YotaDevices',
 'Model': 'YOTA 3+'}

In [520]:
# After calibration, verify focal length makes sense
fx = K_cp[0, 0]
fy = K_cp[1, 1]
cx = K_cp[0, 2]
cy = K_cp[1, 2]

img_width, img_height = img_shape_cv

# Horizontal field of view
import math
hfov_deg = 2 * math.atan(img_width / (2 * fx)) * 180 / math.pi
vfov_deg = 2 * math.atan(img_height / (2 * fy)) * 180 / math.pi

# High fx is CORRECT for telephoto!
# fx = 52,994 for 77mm equiv lens on 4000px wide sensor is realistic

print(f"Focal length: fx={fx:.1f}, fy={fy:.1f}")
print(f"Principal point: cx={cx:.1f}, cy={cy:.1f}")
print(f"Horizontal FOV: {hfov_deg:.1f}°") # Should be ~4-8° for telephoto
print(f"Vertical FOV: {vfov_deg:.1f}°")

# Expected values for phone camera:
# fx/fy: 2000-6000 pixels
# cx ≈ width/2 = 2000
# cy ≈ height/2 = 1500
# HFOV: 60-80°
# VFOV: 45-60°

# What lens focal length does your fx correspond to?
# Assuming 6.4mm sensor width (typical phone)
#focal_length_mm = 3.9 #   "FocalLength35efl": "3.9 mm"
#sensor_width_mm = focal_length_mm * img_width / fx
#print(f"Equivalent sensor width: {sensor_width_mm:.1f}mm")
#print(f"Equivalent focal length: {focal_length_mm:.1f}mm")
#print(f"35mm equivalent: ~{focal_length_mm * 5.6:.0f}mm")
# Should match your camera specs!

Focal length: fx=22148.3, fy=40322.9
Principal point: cx=1808.2, cy=2117.7
Horizontal FOV: 7.7°
Vertical FOV: 5.7°
Equivalent sensor width: 0.5mm
Equivalent focal length: 3.9mm
35mm equivalent: ~22mm


In [507]:
def measure_physical_pattern(blob_data_item, pattern_spacing_m=GRID_BASELINE_M, ratio_threshold = 100):
    """Measure what the pattern should actually be"""

    # Load an image with detected pattern
    img = cv2.imread('calibrate2/IMG_20251107_131104.jpg')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect pattern
    centers = blob_data_item['centers']
    #cv2.findCirclesGrid(gray, (4, 11),
                      #                  flags=cv2.CALIB_CB_ASYMMETRIC_GRID,
                       #                 blobDetector=blobDetector)




    # Reshape to grid (4 cols × 11 rows)
    centers_2d = centers.reshape(11, 4, 2)

    # Measure horizontal spacing in pixels
    row0 = centers_2d[0]  # First row (even)
    row1 = centers_2d[1]  # Second row (odd)

    h_spacing_row0 = np.mean(np.diff(row0[:, 0]))  # X differences
    h_spacing_row1 = np.mean(np.diff(row1[:, 0]))  # X differences

    ratio = h_spacing_row0 / h_spacing_row1

    #print(f"Total points detected: {len(centers)}")

    #print(f"\nHorizontal spacing in pixels:")
    #print(f"  Row 0 (even): {h_spacing_row0:.1f} px")
    #print(f"  Row 1 (odd): {h_spacing_row1:.1f} px")
    #print(f"  Ratio: {ratio:.3f} (should be ~1.0)")

    # Measure vertical spacing
    col0 = centers_2d[:, 0]  # First column
    v_spacing = np.mean(np.diff(col0[:, 1]))
    #print(f"  Vertical spacing: {v_spacing:.1f} px")

    # Check offset between rows
    x_offset = np.mean(row1[:, 0] - row0[:, 0])
    #print(f"  X-offset (row1 vs row0): {x_offset:.1f} px")
    #print(f"  Expected: {h_spacing_row0/2:.1f} px (half horizontal spacing)")

    # Get distance from calibration
    idx = 0  # First image
    distance_m = np.linalg.norm(meta['tvecs'][idx])
    fx = meta['fx']

    #print(f"\nUsing calibration:")
    #print(f"  Distance: {distance_m:.2f}m")
    #print(f"  fx: {fx:.1f}")

    # Calculate physical spacing from pixels
    physical_h_spacing = (h_spacing_row0 * distance_m) / fx
    physical_v_spacing = (v_spacing * distance_m) / fx

    #print(f"\nImplied physical spacing:")
    #print(f"  Horizontal: {physical_h_spacing*1000:.1f}mm")
    #print(f"  Vertical: {physical_v_spacing*1000:.1f}mm")
    #print(f"  Your pattern_spacing_m: {pattern_spacing_m*1000:.1f}mm")

    # What fx would give correct spacing?
    correct_fx = (h_spacing_row0 * distance_m) / (pattern_spacing_m * 2)
    #print(f"\nFor correct spacing {pattern_spacing_m*1000:.1f}mm:")
    #print(f"  Required fx: {correct_fx:.1f}")
    #print(f"  Your fx: {fx:.1f}")
    #print(f"  Ratio: {fx/correct_fx:.2f}x")
    fxratio = fx / correct_fx
    if fxratio > ratio_threshold:
      print(f"===> {blob_data_item['path']}; ratio: {ratio:.3f}; fx_ratio: {fxratio:.3f} ===")



In [521]:

# if there is output remove this files and re-run calibration again
#  notice that calibration error stabilizes
for idx, blob_data_item in enumerate(good_blobs):
  measure_physical_pattern(blob_data_item)

## calibration runs

### auto threshold
Using mean error as threshold: 4.981533362277715

Removing 38 high-error images
Refined calibration RMS: 22.813
Using 37/75 images

=== Calibration Result ===
Reprojection error: 22.812518091102515

Camera matrix:
 [[7.29596271e+04 0.00000000e+00 1.88138659e+03]
 [0.00000000e+00 3.47797510e+04 1.61575910e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Camera matrix f_px:
 72959.62705672771

Distortion coefficients:
 [-4.10806796e+01  1.07852490e+04  1.54370938e+00 -7.27712178e-02
 -1.13709654e+03]

###
 Overall statistics:
  Min err: 1
  Max err: 13
  Median err: 5
  Mean err: 5

Removing 2 high-error images
Refined calibration RMS: 33.952
Using 73/75 images

=== Calibration Result ===
Reprojection error: 33.95160058780786

Camera matrix:
 [[5.23258991e+04 0.00000000e+00 1.71057536e+03]
 [0.00000000e+00 2.92256451e+04 3.35768664e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Camera matrix f_px:
 52325.89913802041

Distortion coefficients:
 [-5.07654855e+01  8.28479284e+02 -1.18910844e+00  1.22746092e-01
  1.91205463e+03]

===========================


In [522]:


save_calibration_yaml(meta, 'yota3_calib_portrait.yaml')


Saved calibration YAML to yota3_calib_portrait.yaml


In [391]:


# ------------------------- Motion stereo helpers -------------------------

def compute_baseline_from_pair(img1_gray: np.ndarray, img2_gray: np.ndarray, K: np.ndarray, dist: np.ndarray) -> Tuple[float, np.ndarray, np.ndarray]:
    """Estimate relative pose between two views using Essential matrix and return baseline magnitude in meters.

    Returns: baseline_m, R, t (unit translation scaled by unknown scale) -- we will compute scale by using board detection if available
    """
    # detect features
    orb = cv2.ORB_create(5000)
    kp1, des1 = orb.detectAndCompute(img1_gray, None)
    kp2, des2 = orb.detectAndCompute(img2_gray, None)

    # match
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)[:2000]

    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches])

    # undistort points
    pts1_ud = cv2.undistortPoints(pts1.reshape(-1,1,2), K, dist)
    pts2_ud = cv2.undistortPoints(pts2.reshape(-1,1,2), K, dist)

    E, mask = cv2.findEssentialMat(pts1_ud, pts2_ud, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    if E is None:
        raise RuntimeError('Essential matrix estimation failed')
    _, R, t, mask_pose = cv2.recoverPose(E, pts1_ud, pts2_ud)

    # t is unit vector up to scale: baseline unknown without external reference.
    # Return baseline in 'relative units' (m) if user provides an external known displacement or scene scale.
    baseline_unit = float(np.linalg.norm(t))
    return baseline_unit, R, t


def estimate_baseline_from_calib_poses(tvecs: List[List[float]]) -> float:
    """If you have a set of tvecs for the same physical calibration board (in meters) from two different frames,
    compute baseline as mean translation magnitude between poses. tvecs expected in meters.
    """
    tvecs_arr = np.array(tvecs)
    # compute pairwise distances
    dists = []
    n = len(tvecs_arr)
    for i in range(n-1):
        for j in range(i+1, n):
            dists.append(np.linalg.norm(tvecs_arr[i] - tvecs_arr[j]))
    if len(dists) == 0:
        raise ValueError('Need at least 2 tvecs')
    return float(np.median(dists))


def undistort_image(img: np.ndarray, K: np.ndarray, dist: np.ndarray):
    h, w = img.shape[:2]
    newK, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), alpha=0)
    und = cv2.undistort(img, K, dist, None, newK)
    return und, newK


# Stereo disparity + reproject to PLY

def compute_disparity_and_pointcloud(
    left_img: np.ndarray,
    right_img: np.ndarray,
    K: np.ndarray,
    dist: np.ndarray,
    R: np.ndarray,
    T: np.ndarray,
    num_disparities: int = 128,
    block_size: int = 3
) -> Tuple[np.ndarray, np.ndarray]:
    """Compute disparity (float32) and 3D points (float32) given stereo extrinsics.

    Returns disparity (H x W) and points_3d (H x W x 3)
    """
    # undistort and rectify
    h, w = left_img.shape[:2]
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(K, dist, K, dist, (w,h), R, T, alpha=0)

    map1x, map1y = cv2.initUndistortRectifyMap(K, dist, R1, P1, (w,h), cv2.CV_32FC1)
    map2x, map2y = cv2.initUndistortRectifyMap(K, dist, R2, P2, (w,h), cv2.CV_32FC1)
    left_rect = cv2.remap(left_img, map1x, map1y, interpolation=cv2.INTER_LINEAR)
    right_rect = cv2.remap(right_img, map2x, map2y, interpolation=cv2.INTER_LINEAR)

    # convert to gray
    left_gray = cv2.cvtColor(left_rect, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_rect, cv2.COLOR_BGR2GRAY)

    # SGBM matcher
    minDisp = 0
    matcher = cv2.StereoSGBM_create(
        minDisparity=minDisp,
        numDisparities=num_disparities,
        blockSize=block_size,
        P1=8*3*block_size**2,
        P2=32*3*block_size**2,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        disp12MaxDiff=1,
        mode=cv2.STEREO_SGBM_MODE_SGBM_3WAY
    )
    disp = matcher.compute(left_gray, right_gray).astype(np.float32) / 16.0

    # optional WLS filter if ximgproc is available
    try:

        # Use left matcher as base
        lmatcher = matcher
        rmatcher = ximgproc.createRightMatcher(lmatcher)
        dispR = rmatcher.compute(right_gray, left_gray).astype(np.float32) / 16.0
        wls = ximgproc.createDisparityWLSFilter(matcher_left=lmatcher)
        wls.setLambda(8000.0)
        wls.setSigmaColor(1.5)
        disp = wls.filter(disp, left_rect, None, dispR)
    except Exception:
        # ximgproc not available: proceed with raw disp
        pass

    # mask invalid disparities
    disp[disp <= 0] = 0

    # Reproject to 3D
    points_3d = cv2.reprojectImageTo3D(disp, Q)
    return disp, points_3d


def save_ply(filename: str, points: np.ndarray, colors: np.ndarray):
    """Save a point cloud (points Nx3, colors Nx3 uint8) to PLY ASCII file."""
    assert points.shape[0] == colors.shape[0]
    with open(filename, 'w') as f:
        f.write('ply\n')
        f.write('format ascii 1.0\n')
        f.write(f'element vertex {points.shape[0]}\n')
        f.write('property float x\nproperty float y\nproperty float z\n')
        f.write('property uchar red\nproperty uchar green\nproperty uchar blue\n')
        f.write('end_header\n')
        for p, c in zip(points, colors):
            f.write(f"{p[0]} {p[1]} {p[2]} {int(c[2])} {int(c[1])} {int(c[0])}\n")
    print(f"Saved PLY: {filename}")


def generate_pointcloud_from_pair(left_color, right_color, K, dist, R, T, out_ply: str):
    disp, pts = compute_disparity_and_pointcloud(left_color, right_color, K, dist, R, T)
    # create mask and flatten
    mask = (disp > 0) & np.isfinite(pts[:,:,2])
    pts_f = pts[mask]
    cols = left_color[mask]
    save_ply(out_ply, pts_f, cols)


# ------------------------- Example usage helper -------------------------

if __name__ == '__main__':
    print('This module is intended to be imported into your notebook. \n')
    print('Example:')
    print("from yota3_calibration_patch import load_gray_images, calibrate_asymmetric_circles, save_calibration_yaml")
    print("imgs = load_gray_images('calib_images')")
    print("meta = calibrate_asymmetric_circles(imgs, pattern_size=(11,8), pattern_spacing_m=0.03)")
    print("save_calibration_yaml(meta, 'yota3_calib.yaml')")

This module is intended to be imported into your notebook. 

Example:
from yota3_calibration_patch import load_gray_images, calibrate_asymmetric_circles, save_calibration_yaml
imgs = load_gray_images('calib_images')
meta = calibrate_asymmetric_circles(imgs, pattern_size=(11,8), pattern_spacing_m=0.03)
save_calibration_yaml(meta, 'yota3_calib.yaml')


### 4. Iesniegšanas kontrolsaraksts
ORTUSā jāiesniedz:
- [ ] Programmas kods (.ipynb)
- [ ] Oriģinālie Middlebury datu kopas attēli (stereo pāris)
- [ ] Disparitātes karte (attēls) bez kalibrācijas
- [ ] Dziļuma karte ar Middlebury kalibrācijas datiem
- [ ] Savi attēli (stereo pāris) un kalibrācijas failu
- [ ] Disparitātes un dziļuma kartes no saviem attēliem
- [ ] Secinājumi (var tikt pievienoti teksta laukā, darba iesniegšanas uzdevumi, vai kā atsēvišķš pdf fails)